# Init

## Install dependencies

In [ ]:
%pip install python-dotenv -q
%pip install SPARQLWrapper -q
%pip install neo4j -q

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

## Connect to Neo4J

In [3]:
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv

load_dotenv()
url = os.getenv('NEO4J_BOLT_URL')
user = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(url, auth=(user,password))

# Populate database

## Purge Neo4j

In [4]:
# Delete all nodes & relationships
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

## Query Wikidata

In [8]:
sparql.setQuery("""
SELECT DISTINCT 
  ?deity ?deityLabel ?deityDescription
#   ?parentDeity ?parentDeityLabel 
  ?domainOfSaint ?domainOfSaintLabel 
  ?deityOf ?deityOfLabel
WHERE 
{
  { ?deity wdt:P31* wd:Q178885 . }     # instance_of deity
  UNION
  {
    ?deity wdt:P31 ?parentDeity .      # deities that are an instance of (P31) a parent deity
    ?parentDeity wdt:P279 wd:Q178885  .  # parent deity that is a "subclass of" (P279) 'deity' (Q178885)
  }

  ?deity p:P31 ?deity_instance .                  # instance_of (Pax is an instance of a Romain Deity)
  OPTIONAL { ?deity_instance pq:P642 ?deityOf . } # "of" attribute of the instance_of (Romain Deity OF peace) 
  OPTIONAL { ?deity wdt:P2925  ?domainOfSaint . } # "domain of saint or deity" (P2925) (Mars is deity of War and Agriculture)
  
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
    
  ?deity rdfs:label ?deityLabel .
  FILTER(LANG(?deityLabel) = "en")    # Filter for English labels
}
ORDER BY ?deityLabel
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(f"Found {len(results['results']['bindings'])} deities")

Found 3924 deities


## Insert into Neo4J

In [6]:
with driver.session() as session:  

  for row in results["results"]["bindings"]:    
      session.run("""
                  MERGE (d:Deity { name: $deity_name, entity: $deity_entity, description: $deity_description })

                  """, 
        deity_name=row["deityLabel"]["value"], 
        deity_entity=row["deity"]['value'],
        deity_description=row.get("deityDescription", {}).get('value', '-')
      )
      
      if ("domainOfSaintLabel" in row):
          session.run("""
              MERGE (deity:Deity { entity: $deity_entity })
              MERGE (domain:Domain { name: $domain_name, entity: $domain_entity }) 
              MERGE (deity)-[:GOD_OF]->(domain)
              """, 
                  domain_name=row["domainOfSaintLabel"]["value"], 
                  domain_entity=row["domainOfSaint"]['value'],
                  deity_entity=row["deity"]['value'])